# Treinamento com interface de alto nível

## Importação das bibliotecas

In [1]:
# http://pytorch.org/
from os.path import exists

import torch

In [12]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from PIL import UnidentifiedImageError
from torch.utils.data import Dataset, random_split

# Dataset

In [3]:
#!/bin/bash
!curl -L -o /content/archive.zip https://www.kaggle.com/api/v1/datasets/download/shaunthesheep/microsoft-catsvsdogs-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  787M  100  787M    0     0  20.9M      0  0:00:37  0:00:37 --:--:-- 22.3M


In [5]:
!unzip -q -o /content/archive.zip -d /content/cats-dogs

In [6]:
cats_dogs_folder = "/content/cats-dogs/PetImages"

In [7]:
transform=transforms.Compose([
    transforms.Resize((100,100)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
    ])

In [10]:
class CustomDataset(datasets.ImageFolder):
  def __getitem__(self, index):
    try:
      return super(CustomDataset, self).__getitem__(index)
    except UnidentifiedImageError:
      print(f"Skipped problematic image {index}")
      return None

In [11]:
dataset = CustomDataset(cats_dogs_folder, transform)

In [13]:
dataset_train, dataset_test = random_split(dataset, [0.7, 0.3])

In [14]:
len(dataset_train)

17500

In [15]:
len(dataset_test)

7500

## DataLoader

In [16]:
def custom_collate_fn(batch):
  batch = list(filter(lambda x: x is not None, batch))
  return torch.utils.data.default_collate(batch)

In [17]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1111)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 500}
test_kwargs = {'batch_size': 100}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_loader = torch.utils.data.DataLoader(dataset_train,**train_kwargs, collate_fn=custom_collate_fn)
test_loader = torch.utils.data.DataLoader(dataset_test, **test_kwargs, collate_fn=custom_collate_fn)

## Criação da rede

In [23]:
input_size = 100*100
output_size = 2
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 2048),
            nn.ReLU(),
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.Linear(4096, 8192),
            nn.ReLU(),
            nn.Linear(8192, 8192),
            nn.ReLU(),
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = x.view(-1, input_size)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()

In [24]:
model

Net(
  (fc): Sequential(
    (0): Linear(in_features=10000, out_features=2048, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2048, out_features=4096, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4096, out_features=8192, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8192, out_features=8192, bias=True)
    (7): ReLU()
    (8): Linear(in_features=8192, out_features=4096, bias=True)
    (9): ReLU()
    (10): Linear(in_features=4096, out_features=2048, bias=True)
    (11): ReLU()
    (12): Linear(in_features=2048, out_features=1024, bias=True)
    (13): ReLU()
    (14): Linear(in_features=1024, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=256, bias=True)
    (17): ReLU()
    (18): Linear(in_features=256, out_features=2, bias=True)
  )
)

## Predict

## Treinamento

In [25]:
model(dataset_train[5][0])

tensor([[-0.6807, -0.7058]], grad_fn=<LogSoftmaxBackward0>)

### Criando o objeto de treinamento

In [26]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [27]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        acc))
    return acc

## Avaliação

In [ ]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 8
scheduler = StepLR(optimizer, step_size=4, gamma=0.7)
best_acc = test(model, device, test_loader)
for epoch in range(1, epochs + 1):
    train(10, False, model, device, train_loader, optimizer, epoch)
    acc = test(model, device, test_loader)
    if acc > best_acc:
      torch.save(model.state_dict(), "catsdogs_nn.pt")
    scheduler.step()


Test set: Average loss: 0.6932, Accuracy: 3770/7500 (50%)

Train Epoch: 1 [0/17500 (0%)]	Loss: 0.693253
Skipped problematic image 8790


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 1 [5000/17500 (29%)]	Loss: 0.693731
Skipped problematic image 14395
Train Epoch: 1 [10000/17500 (57%)]	Loss: 0.689854
Train Epoch: 1 [15000/17500 (86%)]	Loss: 0.691854

Test set: Average loss: 0.6805, Accuracy: 4078/7500 (54%)

Train Epoch: 2 [0/17500 (0%)]	Loss: 0.671882
Train Epoch: 2 [5000/17500 (29%)]	Loss: 0.688344
Skipped problematic image 14395
Train Epoch: 2 [10000/17500 (57%)]	Loss: 0.685100
Skipped problematic image 8790


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 2 [15000/17500 (86%)]	Loss: 0.668594

Test set: Average loss: 0.6657, Accuracy: 4491/7500 (60%)

Train Epoch: 3 [0/17500 (0%)]	Loss: 0.658664
Train Epoch: 3 [5000/17500 (29%)]	Loss: 0.642683
Skipped problematic image 14395
Train Epoch: 3 [10000/17500 (57%)]	Loss: 0.671057
Skipped problematic image 8790
Train Epoch: 3 [15000/17500 (86%)]	Loss: 0.662643


...

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 6 [5000/17500 (29%)]	Loss: 0.575954
Skipped problematic image 14395
Train Epoch: 6 [10000/17500 (57%)]	Loss: 0.571234
Skipped problematic image 8790
Train Epoch: 6 [15000/17500 (86%)]	Loss: 0.588991

Test set: Average loss: 0.6786, Accuracy: 4564/7500 (61%)

Train Epoch: 7 [0/17500 (0%)]	Loss: 0.587328
Skipped problematic image 8790
Train Epoch: 7 [5000/17500 (29%)]	Loss: 0.553419


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Skipped problematic image 14395
Train Epoch: 7 [10000/17500 (57%)]	Loss: 0.598868
Train Epoch: 7 [15000/17500 (86%)]	Loss: 0.561668


KeyboardInterrupt: 

In [ ]:
aux = torch.load("mnist_cnn.pt")

<ipython-input-19-cb91ad9ab067>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aux = torch.load("mnist_cnn.pt")


In [ ]:
aux

OrderedDict([('layer1.weight',
              tensor([[-0.0003, -0.0271,  0.0240,  ...,  0.0353,  0.0151, -0.0153],
                      [-0.0208, -0.0123,  0.0298,  ...,  0.0074, -0.0148, -0.0048],
                      [ 0.0010, -0.0267, -0.0105,  ...,  0.0171,  0.0191,  0.0073],
                      ...,
                      [-0.0194,  0.0303,  0.0269,  ...,  0.0003,  0.0118, -0.0055],
                      [ 0.0055, -0.0117, -0.0062,  ..., -0.0226, -0.0015, -0.0182],
                      [ 0.0190, -0.0259, -0.0127,  ..., -0.0030,  0.0031, -0.0264]],
                     device='cuda:0')),
             ('layer1.bias',
              tensor([-0.0177, -0.0261, -0.0074,  ..., -0.0339, -0.0301,  0.0227],
                     device='cuda:0')),
             ('layer2.weight',
              tensor([[-0.0271, -0.0070, -0.0012,  ..., -0.0216,  0.0065, -0.0299],
                      [-0.0017,  0.0243,  0.0073,  ...,  0.0054, -0.0276,  0.0368],
                      [-0.0211, -0.0065,  0.00

In [ ]:
model.load_state_dict(aux)

<All keys matched successfully>